In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/AAA_MLIP

/content/drive/MyDrive/AAA_MLIP


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity

In [17]:
df=pd.read_csv('books_1.csv')

In [20]:
def encode_genres(df, column_name):
    """
    Encode genres in a DataFrame column using label encoding.

    Parameters:
        df (DataFrame): The DataFrame containing the column to be encoded.
        column_name (str): The name of the column containing genres as a string.

    Returns:
        DataFrame: The DataFrame with an additional column for encoded genres.
    """
    # Initialize LabelEncoder
    label_encoder = LabelEncoder()

    # Split the string by comma and remove whitespace, then encode the genres
    df['Genres_encoded'] = df[column_name].apply(lambda x: label_encoder.fit_transform([genre.strip() for genre in x.split(',')]))

    return df


In [21]:
df = encode_genres(df, 'Genres')
df.head(5)

,Title,Author,Year,Genres,Summary,cover_image,Genres_encoded
0,The Plague,Albert Camus,1947,"Existentialism, Fiction, Absurdist fiction, Novel",The text of The Plague is divided into five p...,http://books.google.com/books/content?id=KVGd-...,"[1, 2, 0, 3]"
1,All Quiet on the Western Front,Erich Maria Remarque,1929-01-29,"War novel, Roman à clef","The book tells the story of Paul Bäumer, a Ge...",http://books.google.com/books/content?id=pgvqs...,"[1, 0]"
2,Blade Runner 3: Replicant Night,K. W. Jeter,1996-10-01,"Science Fiction, Speculative fiction","Living on Mars, Deckard is acting as a consul...",http://books.google.com/books/content?id=ccl_N...,"[0, 1]"
3,Crash,J. G. Ballard,1973,"Speculative fiction, Fiction, Novel",The story is told through the eyes of narrato...,http://books.google.com/books/content?id=xr06D...,"[2, 0, 1]"
4,Dracula,Bram Stoker,1897,"Science Fiction, Speculative fiction, Horror, ...","The novel is told in epistolary format, as a ...",http://books.google.com/books/content?id=8U49A...,"[7, 8, 4, 5, 6, 0, 1, 2, 3]"


In [26]:
from sklearn.preprocessing import MultiLabelBinarizer

# Initialize MultiLabelBinarizer
mlb = MultiLabelBinarizer()

# Fit and transform the genres_encoded column to one-hot encoded genres
genres_encoded_onehot = mlb.fit_transform(df['Genres_encoded'])

# Now, genres_encoded_onehot is a 2D NumPy array suitable for use with KNN


In [29]:
from sklearn.neighbors import NearestNeighbors

# Initialize the KNN model
knn = NearestNeighbors(n_neighbors=5, algorithm='ball_tree')

# Fit the model on the one-hot encoded genres data
knn.fit(genres_encoded_onehot)


NearestNeighbors(algorithm='ball_tree')

In [31]:

def recommend_books(book_title, df, knn_model, mlb, n_recommendations=5):
    # Find the index of the book with the given title
    book_idx = df.index[df['Title'] == book_title].tolist()

    if not book_idx:
        return "Book title not found."
    book_idx = book_idx[0]  # Assuming unique titles, take the first

    # Find the one-hot encoded genres of the given book
    book_genres_encoded = genres_encoded_onehot[book_idx].reshape(1, -1)

    # Use KNN to find similar books
    distances, indices = knn_model.kneighbors(book_genres_encoded, n_neighbors=n_recommendations + 1)

    # Exclude the input book itself and get indices of similar books
    similar_books_indices = indices[0][1:]  # Skip the first index, which is the book itself


    # Filter to exclude the input book itself from the recommendation results
    similar_books_indices = [idx for idx in indices[0] if idx != book_idx][:n_recommendations]


    # Fetch similar books details
    similar_books = df.iloc[similar_books_indices][['Title', 'Author', 'cover_image']]

    return similar_books





In [32]:
# Example: Recommend books similar to "The Plague" based on its genre
book_title = "The Plague"
recommended_books = recommend_books(book_title, df, knn, mlb, n_recommendations=5)
print(recommended_books)

                            Title              Author  \
25              Lord of the Flies     William Golding   
20  The Hound of the Baskervilles  Arthur Conan Doyle   
37                    Blood Music           Greg Bear   
41                    Space Cadet  Robert A. Heinlein   
36               The Forge of God           Greg Bear   

                                          cover_image  
25  http://books.google.com/books/content?id=3C-4d...  
20  http://books.google.com/books/content?id=xT-Uv...  
37  http://books.google.com/books/content?id=ZK0fA...  
41  http://books.google.com/books/content?id=x6ljS...  
36  http://books.google.com/books/content?id=SjUjA...  
